A CSV file containing cleaned Uber data can be found here: 

https://drive.google.com/file/d/1pWGv84eaEZF495H7HVwDOFHnLj66-Nbh/view?usp=sharing

In [1]:
import pandas as pd

In [227]:
uber = pd.read_csv('uber.csv', index_col = 0)
busDf = pd.read_csv('bus.csv', index_col = 0)

/Users/lucaslyon/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [77]:
busDf.tail()

,SERVICE_DATE,PEAK_OFFPEAK_IND,MODE_TYPE,ROUTE_OR_LINE,ROUTE_TYPE,STOP,METRIC_TYPE,OTP_NUMERATOR,OTP_DENOMINATOR,PERCENT_ONTIME
198185,3/4/18 0:00,Off-Peak Service (All Other Times),Bus,45,OTH,NaN,Headway / Schedule Adherence,144,301,0.478405
198186,3/4/18 0:00,Off-Peak Service (All Other Times),Bus,99,OTH,NaN,Headway / Schedule Adherence,115,198,0.580808
198187,3/4/18 0:00,Off-Peak Service (All Other Times),Bus,109,OTH,NaN,Headway / Schedule Adherence,150,293,0.511945
198188,3/4/18 0:00,Off-Peak Service (All Other Times),Bus,70,OTH,NaN,Headway / Schedule Adherence,441,685,0.643796
198189,3/4/18 0:00,Off-Peak Service (All Other Times),Bus,95,OTH,NaN,Headway / Schedule Adherence,78,117,0.666667


In [78]:
lines = busDf.ROUTE_OR_LINE.unique()
lines.tolist()
busDf.loc[(busDf['ROUTE_OR_LINE'] == '100') & (busDf['OTP_NUMERATOR']/busDf['OTP_DENOMINATOR'] < 0.25)]
timesDf = pd.read_csv("times.csv", names=["start", "stop", "time"])
#timesDf.loc[(timesDf['start'] == 837)]
#len(timesDf)

In [79]:
#uber = uber.sort_values(['sourceid', 'dstid'])

In [214]:
slower = busDf.loc[(busDf['ROUTE_OR_LINE'] == '7') & (busDf['OTP_NUMERATOR']/busDf['OTP_DENOMINATOR'] < 0.50)]
slower = slower.drop(['PEAK_OFFPEAK_IND', 'MODE_TYPE', 'ROUTE_OR_LINE', 'STOP', 'ROUTE_TYPE', 'METRIC_TYPE', 'OTP_NUMERATOR', 'OTP_DENOMINATOR'], axis=1)
dates = slower['SERVICE_DATE'].tolist()
percents = slower['PERCENT_ONTIME'].tolist()
conc = []
for j in range(len(dates)):
    conc.append((dates[j], percents[j]))
conc

[('4/22/16 0:00', 0.4965034965034965),
 ('6/16/16 0:00', 0.49264705882352944),
 ('3/11/17 0:00', 0.48444444444444446)]

In [108]:
thisLine = busDf.loc[(busDf['ROUTE_OR_LINE'] == '7')]
averageOnTime = (thisLine['PERCENT_ONTIME']).mean()
averageOnTime
#slowerThanAverage = thisLine.loc[(thisLine['PERCENT_ONTIME'] < averageOnTime)]


0.76912814724468

In [215]:
def lateDays(lines, busDf):
    
    linesSlowerThanAvg = {}
    
    for line in lines:
        #get the piece of the dataframe that has data for each line
        thisLine = busDf.loc[busDf['ROUTE_OR_LINE'] == line]
        
        averageOnTime = ((thisLine['PERCENT_ONTIME']).mean())*0.8
        
        slowerThanAverage = thisLine.loc[(thisLine['PERCENT_ONTIME'] < averageOnTime)]
        
        datesSlowerThanAverage = slowerThanAverage['SERVICE_DATE']
        
        percentOnTime = slowerThanAverage['PERCENT_ONTIME']
        
        listDates = datesSlowerThanAverage.tolist()
        
        listPercents = percentOnTime.tolist()
        
        concatenated = [] 
        
        for ii in range(len(listDates)):
            concatenated.append((listDates[ii], listPercents[ii]))
        
        linesSlowerThanAvg[line] = concatenated
        
    return linesSlowerThanAvg

In [216]:
daysSlowerThanAvg = lateDays(lines, busDf)

In [217]:
#checking how many values each key has
for key, value in daysSlowerThanAvg.items():
    #print value
    print(key, len([item for item in value if item]))

100 229
171 233
225 64
201 199
230 234
60 156
62 93
65 86
78 168
104 174
215 302
26 261
411 279
442 96
449 392
88 80
SL5 0
4 306
132 109
170 352
424 306
426 163
434 172
7 23
106 194
35 99
43 194
44 99
51 152
57A 81
76 124
CT3 268
SL1 0
111 0
114 314
9 33
116 11
222 144
236 276
326 231
39 0
435 252
119 214
21 63
216 39
428 321
451 169
456 126
459 236
503 172
80 188
87 156
SL2 5
8 247
137 329
14 206
220 116
24 187
351 260
554 197
91 325
99 285
501 66
221 277
431 71
448 304
553 202
69 62
92 151
455 204
68 159
746 24
75 124
5 296
11 93
112 288
16 206
22 0
34 63
430 205
502 82
89 136
90 229
109 170
192 194
194 115
214 96
240 242
245 195
441 109
55 156
77 0
117 9
131 221
17 201
211 228
350 133
36 160
436 109
10 98
105 343
108 215
23 0
29 289
31 50
32 0
352 225
45 111
47 159
93 108
210 287
465 144
505 123
71 1
79 174
59 228
72 81
136 253
504 210
558 295
70 116
86 93
202 294
439 163
50 208
57 0
84 266
19 182
30 224
85 162
CT1 270
101 51
217 240
33 183
450 190
212 309
354 162
70A 206
73 2
94 16

In [223]:
daysSlowerThanAvg['1']

[('11/24/16 0:00', 0.545144804088586),
 ('12/25/16 0:00', 0.5582978723404255),
 ('4/28/17 0:00', 0.5570175438596491),
 ('5/25/17 0:00', 0.5602836879432624),
 ('10/8/17 0:00', 0.5149384885764499),
 ('12/25/17 0:00', 0.5618789521228545)]

In [33]:
#uber.loc[(uber['sourceid'] == 885) & (uber['dstid'] == 883)]

In [229]:
#sources = uber.sourceid.unique()
#dests = uber.dstid.unique()
#dests = uber['dstid']
#dests.unique()
uber.loc[uber['sourceid'] >= 1247]
#dests.loc[dests.unique() == 1247]
#1247 in dests
#1246 in sources

,sourceid,dstid,hod,geometric_mean_travel_time,geometric_standard_deviation_travel_time
68751,1247,16,14,1213.03,1.15
112148,1247,16,5,922.09,1.15
212100,1247,16,23,1089.65,1.12
831027,1247,16,22,993.74,1.16
1384784,1247,16,21,1121.90,1.42
1638406,1247,16,7,1679.27,1.35
2179509,1247,16,8,1400.82,1.33
2540589,1247,16,20,1161.16,1.18
258146,1247,16,21,1291.49,1.21
557411,1247,16,2,1115.22,1.21


In [28]:
def averageTravelTimes(sources, dests, dataset):
    fromtotimes = {}
    counter = 0
    for source in sources:
        sliced = dataset.loc[(dataset['sourceid'] == source)]
        for dest in dests:
            furthersliced = sliced.loc[(sliced['dstid'] == dest)]
            if len(furthersliced.index != 0):
                mean = furthersliced['geometric_mean_travel_time'].mean()
                fromtotimes[(source, dest)] = mean
                counter+=1
                if (counter % 5000) == 0:
                    print(source, dest, mean)
    return fromtotimes

In [83]:
uber.loc[(uber['sourceid'] == 837) & (uber['dstid'] == 588)]

,Unnamed: 0,sourceid,dstid,hod,geometric_mean_travel_time,geometric_standard_deviation_travel_time
17448516,25244,837,588,17,106.54,2.12
17448517,263451,837,588,8,106.04,2.49
17448518,393972,837,588,12,91.42,2.14
17448519,501558,837,588,3,79.73,2.37
17448520,559808,837,588,21,82.54,2.33
17448521,784332,837,588,7,108.67,2.51
17448522,813981,837,588,16,87.92,2.52
17448523,1068837,837,588,9,82.02,2.64
17448524,1096994,837,588,18,95.54,2.22
17448525,1142511,837,588,0,68.07,2.39


In [85]:
times = averageTravelTimes(sources, dests, uber)

28 335 1437.2644375
51 896 832.3657591623033
74 703 1111.0325
103 396 552.4605384615386
136 787 1433.2877235772362
155 406 1540.8142708333332
197 798 434.9667391304348
212 836 1296.684081632653
239 1030 921.7447619047617
253 470 2228.1201754385966
286 60 1978.0738709677414
301 1038 2068.685
329 878 989.7792187500003
362 1123 1475.8556684491975
391 311 898.7421428571432
420 316 842.8506806282729
442 427 1310.7316666666666
474 92 1762.24
497 45 2035.6657142857143
510 492 1672.2080000000003
534 1038 2022.3785393258433
584 911 1140.2632692307693
631 504 2250.041666666667
678 1022 823.7022727272725
722 1036 1263.7993251533742
758 883 1181.684479166667
774 915 1681.3025547445254
798 1189 2549.448518518519
818 1163 1685.2320481927711
834 495 1499.5423888888886
856 880 236.48281250000005
869 1227 835.9031770833338
881 503 714.0043229166668
888 713 633.6205729166666
903 297 964.8234374999996
927 719 1322.5235106382977
999 200 138.3086979166667
1013 596 2285.812331288343
1039 1024 707.1698437499

In [230]:
times[(885, 883)]

NameError: name 'times' is not defined

In [103]:
#timesSeries.index.name = 'To From Pair'

In [104]:
#timesSeries.reset_index()
#timesSeries.to_csv('times.csv')

In [231]:
timesDf

,start,stop,time
0,1,12,200.023081
1,1,55,629.978875
2,1,176,180.314952
3,1,265,746.756667
4,1,358,314.217778
5,1,379,488.798469
6,1,603,691.993000
7,1,607,1196.770000
8,1,610,469.765667
9,1,611,164.255278


In [ ]:
zones_100 = ["837", "588", "838", "836", "203", "587"]
zones_7 = ["885", "883", "530", "425", "501"]
zones_1 = ["806", "804", "711", "709", "708", "10402", "10401", "10101", ]